In [1]:
import duckdb
import pyarrow as pa
import pandas as pd
from sklearn.cluster import KMeans

# Establish a connection to DuckDB (in-memory)
con = duckdb.connect(database=':memory:')

# Step 1: Install and load the PostgreSQL extension
con.execute("INSTALL postgres;")
con.execute("LOAD postgres;")

# Step 2: Load data from PostgreSQL into DuckDB
# Replace with your PostgreSQL connection details
postgres_conn_str = "host=localhost port=5432 dbname=postgres user=postgres password=password"
con.execute(f"""
    ATTACH '{postgres_conn_str}' AS postgres_db (TYPE postgres);
""")

# Step 3: Define a Python function for KMeans clustering
def kmeans_cluster(df: pd.DataFrame) -> pd.Series:
    kmeans = KMeans(n_clusters=3)
    clusters = kmeans.fit_predict(df[['latitude', 'longitude', 'numberNegativeNews']])
    return pd.Series(clusters, name='cluster')

# Step 4: Register the Python function as a UDF in DuckDB
con.create_function('kmeans_cluster', kmeans_cluster, [float, float, float], int)

# Step 5: Execute a SQL query using the UDF
result = con.execute("""
    SELECT kmeans_cluster(latitude, longitude, numberNegativeNews) AS cluster
    FROM your_table
""").fetchall()

# Step 6: Process the result
print(result)


InvalidInputException: Invalid Input Error: 3 types provided, but the provided function takes 1 parameters

In [ ]:
from sqlglot.executor import execute
from sqlglot.executor import PythonExecutor


tables = {
    "sushi": [
        {"id": 1, "price": 1.0},
        {"id": 2, "price": 2.0},
        {"id": 3, "price": 3.0},
    ],
    "order_items": [
        {"sushi_id": 1, "order_id": 1},
        {"sushi_id": 1, "order_id": 1},
        {"sushi_id": 2, "order_id": 1},
        {"sushi_id": 3, "order_id": 2},
    ],
    "orders": [
        {"id": 1, "user_id": 1},
        {"id": 2, "user_id": 2},
    ],
}

def kmeans_cluster(latitudes, longitudes, n_clusters=2):
    import numpy as np
    X = np.column_stack((latitudes, longitudes))
    model = KMeans(n_clusters=n_clusters, random_state=42)
    return model.fit_predict(X)

# Step 3: Set up the execution environment
env = {
    'KMEANS_CLUSTER': kmeans_cluster  # Register your function
}



execute(
    """
    SELECT
    KMEANS_CLUSTER(user_id, id) from orders
    """,tables={'my_table': tables})

ExecuteError: Step 'Scan: orders (126918151059792)' failed: name 'KMEANS_CLUSTER' is not defined

In [8]:
import pandas as pd
from sklearn.cluster import KMeans
from sqlglot import parse_one
from sqlglot.executor import PythonExecutor

# Step 1: Prepare your data
data = pd.DataFrame({
    'latitude': [40.7128, 34.0522, 41.8781],
    'longitude': [-74.0060, -118.2437, -87.6298],
    'numberNegativeNews': [5, 3, 4]
})

# Step 2: Define your custom KMeans function
def kmeans_cluster(latitudes, longitudes, negatives, n_clusters=2):
    import numpy as np
    X = np.column_stack((latitudes, longitudes, negatives))
    model = KMeans(n_clusters=n_clusters, random_state=42)
    return model.fit_predict(X)

# Step 3: Set up the execution environment
env = {
    'KMEANS_CLUSTER': kmeans_cluster  # Register your function
}

executor = PythonExecutor(env=env, tables={'my_table': data})

executor.execute("SELECT * from my_table")


AttributeError: 'str' object has no attribute 'leaves'